# Strategy analysis example

Debugging a strategy can be time-consuming. Freqtrade offers helper functions to visualize raw data.
The following assumes you work with SampleStrategy, data for 5m timeframe from Binance and have downloaded them into the data directory in the default location.

In [ ]:
#
# Initialization
#
import os
import sys

import ipynbname
from pathlib import Path

# Set notebook's src module path. Note that you may have to update your IDE's project settings to do the same for the
#  local library imports to work the same

MODULE_PATH = ipynbname.path().parent.parent.parent.parent
sys.path.append(str(MODULE_PATH))

# Keep paths consistent throughout notebook
os.chdir(MODULE_PATH)

# This should always be `./src`
print(f"Current working directory [{os.getcwd()}]")

# Place all local artifacts in a disposable, git-ignored directory
local_artifact_dir = Path(os.getcwd()).parent / "out"
local_artifact_dir.mkdir(parents=True, exist_ok=True)

# Autoreload imports at the beginning of cell execution.
#  https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
#
# Setup utils
#

from utils.logger_util import LoggerUtil
from utils.utils import Utils
from utils import config

LOGGER = LoggerUtil(config.MODEL_ID, local_artifact_dir / "logs")
UTILS = Utils(LOGGER)

UTILS.describe_env()

In [ ]:
from freqtrade.configuration import Configuration

from utils import config

# First prime the user_data_dir key. This will take priority when merged with config.json
freqtrade_config = Configuration({"user_data_dir": config.FREQTRADE_USER_DATA_DIR})
freqtrade_config = freqtrade_config.load_from_files([str(config.FREQTRADE_USER_DATA_DIR / "config.json")])
freqtrade_config["user_data_dir"] = config.FREQTRADE_USER_DATA_DIR

In [ ]:
# Load data using values set above
from freqtrade.data.history import load_pair_history

candles = load_pair_history(
    datadir=config.FREQTRADE_USER_DATA_DIR / "data" / "binance",
    timeframe=freqtrade_config["timeframe"],
    pair=config.CRYPTO_PAIR)

# Confirm success
print("Loaded " + str(len(candles)) + f" rows of data for {config.CRYPTO_PAIR} from {config.FREQTRADE_USER_DATA_DIR}")
candles.head()

## Load and run strategy
* Rerun each time the strategy file is changed

In [ ]:
import matplotlib.pyplot as plt

# Load strategy using values set above
from freqtrade.resolvers import StrategyResolver

import data_processing.marshal_features as mf

# candles_df = candles.tail(50).copy()
candles_df = candles.copy()
strategy = StrategyResolver.load_strategy(freqtrade_config)

# Generate buy/sell signals using strategy
df = strategy.analyze_ticker(candles_df, {'pair': config.CRYPTO_PAIR})

print("--------")
print(df["volume_bin"].value_counts())
print("--------")
print(df["volume"].describe())
print("--------")
print(df.head())
print("--------")

df_plot = df.drop(['sell', 'buy'], axis=1)
ax = df_plot.plot(kind='scatter', x='volume_bin', y='date', figsize=(15, 15))
# [plt.axvline(x) for x in mf.NATURAL_VOLUME_BREAKS]

# # This takes some time to generate
# from pandas.plotting import scatter_matrix
# scatter_matrix(df_plot, alpha=0.2, figsize=(12, 12), diagonal='kde')

ax = df_plot.plot(kind='scatter', x='volume_bin', y='pattern_detected', figsize=(15, 15))
ax = df_plot.plot(kind='scatter', x='pattern_count', y='pattern_detected', figsize=(15, 15))
# ax = df_plot.plot(kind='scatter', x='volume', y='pattern_detected', figsize=(15, 15))

### Display the trade details

* Note that using `data.head()` would also work, however most indicators have some "startup" data at the top of the dataframe.
* Some possible problems
    * Columns with NaN values at the end of the dataframe
    * Columns used in `crossed*()` functions with completely different units
* Comparison with full backtest
    * having 200 buy signals as output for one pair from `analyze_ticker()` does not necessarily mean that 200 trades will be made during backtesting.
    * Assuming you use only one condition such as, `df['rsi'] < 30` as buy condition, this will generate multiple "buy" signals for each pair in sequence (until rsi returns > 29). The bot will only buy on the first of these signals (and also only if a trade-slot ("max_open_trades") is still available), or on one of the middle signals, as soon as a "slot" becomes available.  


In [ ]:
# Report results
print(f"Generated {df['buy'].sum()} buy signals")
data = df.set_index('date', drop=False)
data.tail()

## Load existing objects into a Jupyter notebook

The following cells assume that you have already generated data using the cli.  
They will allow you to drill deeper into your results, and perform analysis which otherwise would make the output very difficult to digest due to information overload.

### Load backtest results to pandas dataframe

Analyze a trades dataframe (also used below for plotting)

In [ ]:
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats

# if backtest_dir points to a directory, it'll automatically load the last backtest file.
backtest_dir = config["user_data_dir"] / "backtest_results"
# backtest_dir can also point to a specific file 
# backtest_dir = config["user_data_dir"] / "backtest_results/backtest-result-2020-07-01_20-04-22.json"

In [ ]:
# You can get the full backtest statistics by using the following command.
# This contains all information used to generate the backtest result.
stats = load_backtest_stats(backtest_dir)

strategy = 'SampleStrategy'
# All statistics are available per strategy, so if `--strategy-list` was used during backtest, this will be reflected here as well.
# Example usages:
print(stats['strategy'][strategy]['results_per_pair'])
# Get pairlist used for this backtest
print(stats['strategy'][strategy]['pairlist'])
# Get market change (average change of all pairs from start to end of the backtest period)
print(stats['strategy'][strategy]['market_change'])
# Maximum drawdown ()
print(stats['strategy'][strategy]['max_drawdown'])
# Maximum drawdown start and end
print(stats['strategy'][strategy]['drawdown_start'])
print(stats['strategy'][strategy]['drawdown_end'])

# Get strategy comparison (only relevant if multiple strategies were compared)
print(stats['strategy_comparison'])


In [ ]:
# Load backtested trades as dataframe
trades = load_backtest_data(backtest_dir)

# Show value-counts per pair
trades.groupby("pair")["sell_reason"].value_counts()

### Load live trading results into a pandas dataframe

In case you did already some trading and want to analyze your performance

In [ ]:
from freqtrade.data.btanalysis import load_trades_from_db

# Fetch trades from database
trades = load_trades_from_db("sqlite:///tradesv3.sqlite")

# Display results
trades.groupby("pair")["sell_reason"].value_counts()

## Analyze the loaded trades for trade parallelism
This can be useful to find the best `max_open_trades` parameter, when used with backtesting in conjunction with `--disable-max-market-positions`.

`analyze_trade_parallelism()` returns a timeseries dataframe with an "open_trades" column, specifying the number of open trades for each candle.

In [ ]:
from freqtrade.data.btanalysis import analyze_trade_parallelism

# Analyze the above
parallel_trades = analyze_trade_parallelism(trades, '5m')

parallel_trades.plot()

## Plot results

Freqtrade offers interactive plotting capabilities based on plotly.

In [ ]:
from freqtrade.plot.plotting import generate_candlestick_graph

# Limit graph period to keep plotly quick and reactive

# Filter trades to one pair
trades_red = trades.loc[trades['pair'] == pair]

data_red = data['2019-06-01':'2019-06-10']
# Generate candlestick graph
graph = generate_candlestick_graph(pair=pair,
                                   data=data_red,
                                   trades=trades_red,
                                   indicators1=['sma20', 'ema50', 'ema55'],
                                   indicators2=['rsi', 'macd', 'macdsignal', 'macdhist']
                                   )




In [ ]:
# Show graph inline
# graph.show()

# Render graph in a seperate window
graph.show(renderer="browser")


Feel free to submit an issue or Pull Request enhancing this document if you would like to share ideas on how to best analyze the data.